In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import imodelsx.metrics
import numpy as np
sys.path.append('../experiments/')
import os
import pandas as pd
import numpy as np
import transformers
import sys
from os.path import join
import datasets
from typing import Dict, List
from dict_hash import sha256
import numpy as np
import openai
openai.api_key = open(os.path.expanduser('~/.openai_api_key')).read().strip()
import imodelsx.treeprompt.stump

from tprompt.compiler.evaluator import PromptHooker, modify_activations
from tprompt.compiler import compiling
import tprompt.compiler

# Load prompt vector

In [47]:
# hyperparams
checkpoint = 'gpt2-xl'
prompts = [
    # " This review of a movie is",
    # " Positive or Negative? The movie was",
    # " The sentiment of the movie was",
    # " The plot of the movie was really",
    # " The acting in the movie was",
    " This movie is",
    " Positive or Negative? The movie was",
    " The sentiment of the movie was",
    " The plot of the movie was really",
    " The acting in the movie was",
]
# note: also requires specifying the layer for the hook below

In [48]:
# instructions here for setting up vec2text
# !pip install vec2text
# import nltk
# nltk.download('punkt')

In [49]:
avg_soft_prompt = compiling.get_avg_soft_prompt(checkpoint, prompts)

In [55]:
import nltk
avg_text = tprompt.compiler.compiling.get_avg_inverted_text_prompt(prompts)

# remove any repeated full senteces in the string


def remove_repeated_sentences(text):
    sentences = nltk.tokenize.sent_tokenize(text)
    sentences = list(dict.fromkeys(sentences))
    return ' '.join(sentences)


avg_text = remove_repeated_sentences(avg_text[0]).strip()
print(repr(avg_text))

/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


'The movie was a movie. The plot was very interesting. The movie was actually quite funny. The movie was filmed in a very realistic manner. The movie is'


# Evaluate accuracies

In [ ]:
dset_train = datasets.load_dataset('rotten_tomatoes')['train']
dset_train = dset_train.select(np.random.choice(
    len(dset_train), size=100, replace=False))
# dset_val = datasets.load_dataset('rotten_tomatoes')['validation']
# dset_val = dset_val.select(np.random.choice(
#     len(dset_val), size=100, replace=False))

  0%|          | 0/3 [00:00<?, ?it/s]

In [52]:
verbalizer = {0: " Negative.", 1: " Positive."}
kwargs = dict(
    checkpoint=checkpoint,
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    prompt_at_start_or_end='end',
    prompt_template="{example}{prompt}",
)
# m = PromptHooker(
#     prompts=[prompts[0]],
#     hook_weights=avg_soft_prompt,
#     **kwargs
# )
# m.fit(dset_train["text"], dset_train["label"])
# acc_avg = m.prompt_accs_[0]
# print('\n**Soft Averaged** ->', acc_avg)
# print('------------------------')


print('------------------------')
print('\n**Inverted avg prompt**')

m = PromptHooker(
    prompts=[avg_text],
    hook_weights=None,
    **kwargs
)
m.fit(dset_train["text"], dset_train["label"])
print('Averaged inverted prompt -> ', m.prompt_accs_[0])


print('------------------------')
print('\n**Individual prompts**')
m = PromptHooker(
    prompts=prompts,
    hook_weights=None,
    **kwargs
)
m.fit(dset_train["text"], dset_train["label"])

accs0 = deepcopy(m.prompt_accs_)
for i, prompt in enumerate(prompts):
    print(i, prompt, '->', accs0[i])

------------------------

Inverted avg prompt
Prompt 0: The movie was a movie. The plot was very interesting. The movie was actually quite funny. The movie was filmed in a very realistic manner. The movie is


Averaged inverted prompt ->  0.46
------------------------

Individual prompts
Prompt 0:  This movie is


Prompt 1:  Positive or Negative? The movie was


Prompt 2:  The sentiment of the movie was


Prompt 3:  The plot of the movie was really


Prompt 4:  The acting in the movie was


Prompt 5:  The movie was a movie.  The plot was a movie.  The actors were really good.  The movie is


                                                                                            5.67it/s]

0  This movie is -> 0.67
1  Positive or Negative? The movie was -> 0.51
2  The sentiment of the movie was -> 0.47
3  The plot of the movie was really -> 0.74
4  The acting in the movie was -> 0.55
5  The movie was a movie.  The plot was a movie.  The actors were really good.  The movie is -> 0.46


# Unit test

In [ ]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained(checkpoint)
tok_lens = [len(tok.encode(prompt)) for prompt in prompts]

In [ ]:
tok_lens

In [ ]:
m = PromptHooker(
    checkpoint=checkpoint,
    # 3 different prompts with same len
    prompts=[prompts[0], prompts[2], prompts[3]],
    verbalizer=verbalizer,
    cache_prompt_features_dir=None,
    random_state=42,
    hook_weights=avg_soft_prompt,
    prompt_at_start_or_end='end',
    prompt_template="{example}{prompt}",
)
m.fit(dset_train["text"], dset_train["label"])

In [ ]:
m.prompt_accs_